In [1]:
from keras import models
from keras import layers
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau
from keras import callbacks
from keras.applications.vgg16 import VGG16

 
# Create the model
model=models.Sequential()

VGG= VGG16(include_top=False,input_shape=(128,128,3),weights='imagenet')

model.add(VGG)
model.add(layers.Flatten())
model.add(layers.Dense(6, activation = "softmax"))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

Using TensorFlow backend.


58892288/58889256 [==============================] - 1s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 49158     
Total params: 14,763,846
Trainable params: 14,763,846
Non-trainable params: 0
_________________________________________________________________


In [2]:
import os
base_dir = '/kaggle/input/duth-cv-2019-2020-hw-4/vehicles'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)
       
val_datagen  = ImageDataGenerator(rescale=1./255)

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    # color_mode='grayscale',
                                                    target_size=(128,128),
                                                    shuffle=True)     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  val_datagen.flow_from_directory(validation_dir,
                                                        batch_size=32,
                                                        class_mode='categorical',
                                                        #  color_mode='grayscale',
                                                         target_size=(128,128)) 











Found 2494 images belonging to 6 classes.
Found 311 images belonging to 6 classes.


In [4]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])
# Train the model
callbacks = callback = callbacks.ModelCheckpoint("best.h5", monitor='val_loss', mode = 'min', verbose=1, save_best_only = True)
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=50,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1,callbacks=[callbacks])
 
# Save the model


Epoch 1/50
78/77 [==============================] - 29s 375ms/step - loss: 1.1265 - acc: 0.5682 - val_loss: 0.9272 - val_acc: 0.6881

Epoch 00001: val_loss improved from inf to 0.92721, saving model to best.h5
Epoch 2/50
78/77 [==============================] - 24s 308ms/step - loss: 0.6611 - acc: 0.7510 - val_loss: 0.6257 - val_acc: 0.7846

Epoch 00002: val_loss improved from 0.92721 to 0.62572, saving model to best.h5
Epoch 3/50
78/77 [==============================] - 24s 306ms/step - loss: 0.5032 - acc: 0.8192 - val_loss: 0.4627 - val_acc: 0.7814

Epoch 00003: val_loss improved from 0.62572 to 0.46275, saving model to best.h5
Epoch 4/50
78/77 [==============================] - 24s 313ms/step - loss: 0.4028 - acc: 0.8549 - val_loss: 0.6409 - val_acc: 0.7685

Epoch 00004: val_loss did not improve from 0.46275
Epoch 5/50
78/77 [==============================] - 24s 311ms/step - loss: 0.3304 - acc: 0.8769 - val_loss: 0.2031 - val_acc: 0.8585

Epoch 00005: val_loss improved from 0.46275

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from keras.preprocessing import image
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import csv

model = tf.keras.models.load_model('/kaggle/working/best.h5')
rowlist = [['Id', 'Category']]

for dirname, _, filenames in os.walk('/kaggle/input/duth-cv-2019-2020-hw-4/vehicles/test'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        img = image.load_img(path, target_size=(128, 128), grayscale=False, interpolation='bilinear')
        
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        
        classes_pred = model.predict(x)
        cls_pred = np.argmax(classes_pred)
        rowlist.append([filename, cls_pred])
        #print(filename, cls_pred)
        with open('output.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(rowlist)
        